In [1]:
!pip install gspread google-auth flask

In [2]:
!pip install pyngrok

In [ ]:
!ngrok authtoken your-token-here

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [31]:
from pyngrok import ngrok

# Open a tunnel to the Flask app
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://e581-34-106-125-28.ngrok-free.app" -> "http://localhost:5000"


In [32]:
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
from flask import Flask, request, jsonify

app = Flask(__name__)

# Google Sheets authentication
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("credentials.json", scopes=scopes)
client = gspread.authorize(creds)

@app.route('/update-attendance', methods=['POST'])
def update_attendance():
    # Connect to the Google Sheet
    data = request.get_json()
    sheet_id = data.get('sheet_id')
    print(f"Received Sheet ID: {sheet_id}")
    workbook = client.open_by_key(sheet_id)
    sheet = workbook.sheet1  # assuming your data is on the first sheet

    # Retrieve header row and data once to minimize read requests
    header_row = sheet.row_values(1)
    all_rows = sheet.get_all_records()  # Fetches the sheet as a list of dictionaries
    date_columns = [i for i, col in enumerate(header_row) if "/" in col]
    latest_date_col = date_columns[-1] + 1  # Latest date column index (1-based)

    # Initialize batch update lists
    attendance_updates = []
    percentage_updates = []

    # Color scheme for formatting
    defaulter_color = {
        "backgroundColor": {"red": 1.0, "green": 1.0, "blue": 0.0, "alpha": 0.2}  # Yellow
    }
    clear_color = {
        "backgroundColor": None  # Clear formatting
    }
    p_color = {
        "backgroundColor": {"red": 0.0, "green": 1.0, "blue": 0.0, "alpha": 0.2}  # Green for Present
    }
    a_color = {
        "backgroundColor": {"red": 1.0, "green": 0.0, "blue": 0.0, "alpha": 0.2}  # Red for Absent
    }

    # First, mark attendance
    for i, row in enumerate(all_rows, start=2):  # start=2 to skip header row
        in_time = row.get('in-time', '')
        out_time = row.get('out-time', '')

        # Check attendance if out_time is missing
        if not out_time:
            presence = "A"
        else:
            if in_time and out_time:
                time_format = "%H:%M:%S"
                in_time_obj = datetime.strptime(in_time, time_format)
                out_time_obj = datetime.strptime(out_time, time_format)
                duration = (out_time_obj - in_time_obj).total_seconds() / 60  # duration in minutes

                # Set threshold for present (e.g., 5 minutes)
                presence = "P" if duration >= 5 else "A"

        # Update cell for the latest date column with "P" or "A"
        attendance_updates.append({
            "range": f"{chr(64 + latest_date_col)}{i}",
            "values": [[presence]]
        })

        # Apply color formatting for P and A
        cell_range = f"{chr(64 + latest_date_col)}{i}"
        if presence == "P":
            sheet.format(cell_range, p_color)
        else:
            sheet.format(cell_range, a_color)

    # Calculate and update attendance percentage
    for i, row in enumerate(all_rows, start=2):
        total_classes = sum(1 for j in date_columns if row[header_row[j]] in ["P", "A"])
        attended_classes = sum(1 for j in date_columns if row[header_row[j]] == "P")
        attendance_percentage = (attended_classes / total_classes) * 100 if total_classes > 0 else 0

        # Update percentage column
        percentage_updates.append({
            "range": f"G{i}",  # Assuming percentage is in column G
            "values": [[f"{attendance_percentage:.2f}%"]]
        })

        # Apply or clear defaulter formatting
        if attendance_percentage <= 50:
            # Highlight row if defaulter
            sheet.format(f"A{i}:G{i}", defaulter_color)
        else:
            # Clear formatting if no longer a defaulter
            sheet.format(f"A{i}:G{i}", clear_color)

    # Apply batch updates
    try:
        if attendance_updates:
            sheet.batch_update(attendance_updates)
        if percentage_updates:
            sheet.batch_update(percentage_updates)
    except Exception as e:
        print(f"Error during batch update: {e}")

    return jsonify({"status": "success"}), 200

In [33]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received Sheet ID: 1ioJr0hBbvyyVmND4WuCwRc7fmJ6e9sCDhcAzIdkT8T4


INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 12:48:31] "POST /update-attendance HTTP/1.1" 200 -


Received Sheet ID: 1ioJr0hBbvyyVmND4WuCwRc7fmJ6e9sCDhcAzIdkT8T4


INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 12:48:45] "POST /update-attendance HTTP/1.1" 200 -
